In [7]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import weasyprint as wp
import jinja2
import calendar

from datahandling import *
from graphing import *

import IPython.core.display as ipd

mpl.style.use('seaborn-bright')#'fivethirtyeight')
mpl.rcParams['lines.linewidth'] = 1
mpl.rcParams['figure.figsize'] = (3,2)
mpl.rcParams['axes.titlesize'] = 'large'
mpl.rcParams['axes.labelsize'] = 'small'
mpl.rcParams['xtick.labelsize'] = 'small'
mpl.rcParams['ytick.labelsize'] = 'small'
mpl.rcParams['legend.fontsize'] = 'small'

In [9]:
df = readfile("./testdata/fetch.bax")
t_start, t_end = (df.index.min(), df.index.max())
dfs = split_by_id(df)
names = unique_sensors(df)

dfs = fix_humidity(dfs)
dfs = diff_pir(dfs)
dfs = fix_temp(dfs)
d=dfs[list(dfs.keys())[0]]

/usr/local/lib/python3.5/site-packages/pandas/core/indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.5/site-packages/pandas/core/indexing.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


In [10]:
%%capture

figs=[]
fig_dir = 'figs'
for day in pd.date_range(t_start, t_end, freq='D', normalize=True):
    start = day
    end =(day + pd.Timedelta('1 day'))
    x_data = [ dfs[i].loc[start:end,].index for i in dfs ]
    y_data = [ dfs[i].loc[start:end, 'Temp'].values for i in dfs ]
    
    # Ignore days with one value (e.g. fetch interface returns 23:59:58 from the previous day)
    if sum( [len(x) for x in x_data] ) <= len( x_data ):
        continue
    
    #files.append('fig-temp-{0}'.format(day.date()))
    figs.append(
        graph(x_data, 
          y_data,
          x_label="Time (Hours)",
          y_label="Temperature",
          y_series=list(dfs.keys()),
          title="{0} {1}".format(calendar.day_name[start.dayofweek], start.date().strftime('%d %b %Y')),
          #colors=graph.colors[::2],
          twin_y=True,
          #legend=True,
          savefig=True,
          savedir='figs')
        )

In [11]:
import base64
print('\n'.join(base64.b64decode(figs[0]).decode('utf8').split('\n')[4:7])[:100])
#ipd.HTML("<img src='data:image/svg+xml;charset=utf-8;base64,%s' />" % figs[0])
ipd.HTML(base64.b64decode(figs[0]).decode('utf8'))

<svg height="144pt" version="1.1" viewBox="0 0 216 144" width="216pt" xmlns="http://www.w3.org/2000/


In [12]:
temperature_template = '''
<h2>{{series}} Report: OpenLab</h1>
<h3>Time period: <em>{{t_start}} to {{t_end}}</em></h3>
{% for n in fig_list %}
<figure>
    <img src="data:image/svg+xml;charset=utf-8;base64,{{n}}" />
    <!--<figcaption></figcaption>-->
</figure>
{% endfor %}
'''

print_css = wp.CSS(string='''
@page { size: A4 portrait; margin:1cm 2cm; }
html { font-family: sans; }
html, body {
  width: 100%;
  margin: 0;
  padding: 0;
  position: relative;
}
figure {
    padding:0;
    margin:0;
    /*position:relative;*/
    display:block;
    float:left;
    width:50%;
}
img { width: 100%; }
''')

debug_css = wp.CSS(string='''
* {
  box-sizing: border-box;
}
* {
  outline: solid 2px #900;
}
* > * {
  outline-color: #090;
}
* > * > * {
  outline-color: #009;
}
''')

variables = {
    't_start': t_start, 
    't_end'  : t_end,
    'fig_list' : figs,
    'series' : 'Temperature'
}
output = jinja2.Template(temperature_template).render(**variables)

In [13]:
# with open('test.htm', 'w+') as t: t.write(output)
#ipd.display(ipd.HTML(output))

# write to PDF
htm = wp.HTML(string=output, base_url='.')
pdf = htm.write_pdf(target='weasyprint_output.pdf', zoom=2, stylesheets=[print_css])#, debug_css])

!open weasyprint_output.pdf